In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [21]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "citibike_tabular_data_28d.parquet")
df.head()



,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 11:00:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,1,5,12,21,6,8,8,2023-01-26 11:00:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,10,12,6,8,3,2023-01-27 11:00:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,6,5,10,23,2023-01-28 11:00:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,1,2,9,6,14,2023-01-29 11:00:00,5329.03


In [22]:
df_est = df.copy()
df_est["pickup_hour"] = pd.to_datetime(df_est["pickup_hour"])
df_est["pickup_hour"] = df_est["pickup_hour"].dt.tz_localize("UTC").dt.tz_convert("America/New_York")


In [23]:
df.head()

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 11:00:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,1,5,12,21,6,8,8,2023-01-26 11:00:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,10,12,6,8,3,2023-01-27 11:00:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,6,5,10,23,2023-01-28 11:00:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,1,2,9,6,14,2023-01-29 11:00:00,5329.03


In [24]:
df_est.head()

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 06:00:00-05:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,1,5,12,21,6,8,8,2023-01-26 06:00:00-05:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,10,12,6,8,3,2023-01-27 06:00:00-05:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,6,5,10,23,2023-01-28 06:00:00-05:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,1,2,9,6,14,2023-01-29 06:00:00-05:00,5329.03


In [25]:
from datetime import datetime
from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(657, 674)
(657,)
(366, 674)
(366,)


In [26]:
import numpy as np

class BaselineModelPreviousHour:
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test["rides_t-1"]


In [27]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)


In [28]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


5.7459


In [29]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
load_dotenv() 

mlflow = set_mlflow_tracking()


INFO:src.experiment_utils:MLflow tracking URI and credentials set.


In [30]:
log_model_to_mlflow(
    model,
    X_test,
    "BaselineModelPreviousHour",
    "mean_absolute_error",
    score=test_mae
)


INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousHour
INFO:src.experiment_utils:Logged mean_absolute_error: 5.745901639344262
/opt/anaconda3/envs/sp25_citibike/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.
Registered model 'BaselineMode

🏃 View run stately-wasp-408 at: https://dagshub.com/rohanjai/sp25_taxi-main.mlflow/#/experiments/0/runs/791562c83a2444d3ac07d2e253d0652e
🧪 View experiment at: https://dagshub.com/rohanjai/sp25_taxi-main.mlflow/#/experiments/0


In [35]:
class BaselineModelPreviousWeek:
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test[f"rides_t-{7*24}"]  # rides_t-168


In [36]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)


In [37]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

9.7951


In [38]:
log_model_to_mlflow(model, X_test, "BaselineModelPreviousWeek", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousWeek
INFO:src.experiment_utils:Logged mean_absolute_error: 9.795081967213115
/opt/anaconda3/envs/sp25_citibike/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.
Registered model 'BaselineMode

🏃 View run delicate-tern-291 at: https://dagshub.com/rohanjai/sp25_taxi-main.mlflow/#/experiments/1/runs/3a4a53185e8d4eff83d3874fac104fe3
🧪 View experiment at: https://dagshub.com/rohanjai/sp25_taxi-main.mlflow/#/experiments/1


In [34]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


9.7951


In [39]:
import numpy as np
import pandas as pd

class BaselineModelLast4Weeks:
    """
    A baseline model that predicts the average of the last 4 weeks (28 days)
    for each test instance.
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass  # No training needed for baseline

    def predict(self, X_test: pd.DataFrame) -> np.array:
        last_4_weeks_columns = [
            f"rides_t-{7*24}",   # 1 week ago
            f"rides_t-{14*24}",  # 2 weeks ago
            f"rides_t-{21*24}",  # 3 weeks ago
            f"rides_t-{28*24}"   # 4 weeks ago
        ]
        for col in last_4_weeks_columns:
            if col not in X_test.columns:
                raise ValueError(f"Missing required column: {col}")
        return X_test[last_4_weeks_columns].mean(axis=1).to_numpy()


In [40]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)


In [41]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


8.2787


In [25]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

4.5444


In [42]:
log_model_to_mlflow(model, X_test, "BaselineModelLast4Weeks", "mean_absolute_error", score=test_mae)


INFO:src.experiment_utils:Experiment set to: BaselineModelLast4Weeks
INFO:src.experiment_utils:Logged mean_absolute_error: 8.278688524590164
/opt/anaconda3/envs/sp25_citibike/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.
Registered model 'BaselineModelL

🏃 View run angry-bird-523 at: https://dagshub.com/rohanjai/sp25_taxi-main.mlflow/#/experiments/2/runs/1e59f3ddfec4421daa544e37bab4edcb
🧪 View experiment at: https://dagshub.com/rohanjai/sp25_taxi-main.mlflow/#/experiments/2


In [44]:
X_test.head()  # Get the total number of rows


,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
219,14,16,19,19,25,46,56,48,29,23,...,0,0,0,4,15,14,13,16,2023-09-01 11:00:00,5329.03
220,45,43,34,49,51,63,63,49,52,29,...,0,0,0,2,7,4,12,32,2023-09-02 11:00:00,5329.03
221,26,32,36,46,62,50,58,48,47,25,...,1,2,0,1,1,12,14,12,2023-09-03 11:00:00,5329.03
222,9,8,19,14,16,24,49,76,49,35,...,0,1,0,1,5,5,17,21,2023-09-04 11:00:00,5329.03
223,8,13,23,15,15,27,66,89,80,50,...,0,0,0,0,0,20,17,12,2023-09-05 11:00:00,5329.03


In [45]:
plot_aggregated_time_series(X_test, y_test, row_id=220, predictions=predictions)
